<a href="https://colab.research.google.com/github/karna-charan/LLM/blob/main/Credit_Card_Fraud_Detection_using_Backpropagation_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

# -----------------------------
# 1. Create Synthetic Dataset
# -----------------------------
np.random.seed(42)

# 10,000 transactions, 30 features
X = np.random.randn(10000, 30)

# Highly imbalanced fraud labels (1% fraud)
y = np.zeros(10000)
y[:100] = 1

# Shuffle dataset
shuffle_idx = np.random.permutation(len(X))
X = X[shuffle_idx]
y = y[shuffle_idx]

# Normalize
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Convert to tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test  = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).view(-1,1)
y_test  = torch.tensor(y_test, dtype=torch.float32).view(-1,1)

# -----------------------------
# 2. Define FFNN Model
# -----------------------------
class FraudNet(nn.Module):
    def __init__(self):
        super(FraudNet, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(30, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

model = FraudNet()

# -----------------------------
# 3. Loss & Optimizer
# -----------------------------
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# -----------------------------
# 4. Training (Backpropagation)
# -----------------------------
epochs = 10

for epoch in range(epochs):
    outputs = model(X_train)
    loss = criterion(outputs, y_train)

    optimizer.zero_grad()
    loss.backward()   # Backpropagation
    optimizer.step()

    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

# -----------------------------
# 5. Evaluation
# -----------------------------
with torch.no_grad():
    preds = model(X_test)
    predicted = (preds > 0.5).float()
    accuracy = (predicted == y_test).float().mean()

print("\nTest Accuracy:", accuracy.item())

# -----------------------------
# 6. Fraud Prediction
# -----------------------------
sample = X_test[0].unsqueeze(0)
fraud_prob = model(sample)

print("Fraud Probability:", fraud_prob.item())


Epoch 1, Loss: 0.5972
Epoch 2, Loss: 0.5917
Epoch 3, Loss: 0.5864
Epoch 4, Loss: 0.5811
Epoch 5, Loss: 0.5759
Epoch 6, Loss: 0.5707
Epoch 7, Loss: 0.5656
Epoch 8, Loss: 0.5605
Epoch 9, Loss: 0.5555
Epoch 10, Loss: 0.5505

Test Accuracy: 0.9909999966621399
Fraud Probability: 0.4320995509624481
